Convert Yelp JSON files to dataframe, explore dataset

In [33]:
import json
import pandas as pd
from glob import glob
import os
import codecs
import itertools

Want to subset cities with many reviews, focus on resturants

In [4]:
with codecs.open('yelp_academic_dataset_user.json', encoding='utf_8') as f:
    first_user_record = f.readline() 

print(first_user_record)

{"user_id": "oMy_rEb0UBEmMlu-zcxnoQ", "name": "Johnny", "review_count": 8, "yelping_since": "2014-11-03", "friends": ["cvVMmlU1ouS3I5fhutaryQ", "nj6UZ8tdGo8YJ9lUMTVWNw", "RTtdEVhAmeWqCSp0IgJ99w", "t3UKA1sl4e6LY_xsjuvI0A", "s057_BvOfnKNvQquJf7VNg", "VYrdepCgdzJ4WaxP7dBGpg", "XXLSk6sQQDyr3dZ4zE-O0g", "Py8ThfExQaXF2Woqr7kWUw", "233YNvzVtZ1ObkaNkUzNIw", "L6iE9NpmHHJQTk0JQlRlSA", "Y7XTMgZ_q5Bj5f9KhK1R4Q"], "useful": 0, "funny": 0, "cool": 0, "fans": 0, "elite": [], "average_stars": 4.67, "compliment_hot": 0, "compliment_more": 0, "compliment_profile": 0, "compliment_cute": 0, "compliment_list": 0, "compliment_note": 0, "compliment_plain": 1, "compliment_cool": 0, "compliment_funny": 0, "compliment_writer": 0, "compliment_photos": 0}



In [5]:
with codecs.open('yelp_academic_dataset_business.json', encoding='utf_8') as f:
    first_business_record = f.readline() 

print(first_business_record)

{"business_id": "FYWN1wneV18bWNgQjJ2GNg", "name": "Dental by Design", "neighborhood": "", "address": "4855 E Warner Rd, Ste B9", "city": "Ahwatukee", "state": "AZ", "postal_code": "85044", "latitude": 33.3306902, "longitude": -111.9785992, "stars": 4.0, "review_count": 22, "is_open": 1, "attributes": {"AcceptsInsurance": true, "ByAppointmentOnly": true, "BusinessAcceptsCreditCards": true}, "categories": ["Dentists", "General Dentistry", "Health & Medical", "Oral Surgeons", "Cosmetic Dentists", "Orthodontists"], "hours": {"Friday": "7:30-17:00", "Tuesday": "7:30-17:00", "Thursday": "7:30-17:00", "Wednesday": "7:30-17:00", "Monday": "7:30-17:00"}}



Get businesses in 'restaurant', 'food', and 'bar' categories

In [43]:
restaurant_ids = set()

# open the businesses file
with codecs.open('yelp_academic_dataset_business.json', encoding='utf_8') as f:
    
    # iterate through each line (json record) in the file
    for business_json in f:
        
        # convert the json record to a Python dict
        business = json.loads(business_json)
        
        # if this business is not a restaurant, skip to the next one
        if u'Restaurants' not in business[u'categories']:
            continue
            
        # add the restaurant business id to our restaurant_ids set
        restaurant_ids.add(business[u'business_id'])


In [20]:
food_ids = set()

# open the businesses file
with codecs.open('yelp_academic_dataset_business.json', encoding='utf_8') as f:
    
    # iterate through each line (json record) in the file
    for business_json in f:
        
        # convert the json record to a Python dict
        business = json.loads(business_json)
        
        # if this business is not a food, skip to the next one
        if u'Food' not in business[u'categories']:
            continue
            
        # add the food business id to our food_ids set
        food_ids.add(business[u'business_id'])


In [21]:
bar_ids = set()

# open the businesses file
with codecs.open('yelp_academic_dataset_business.json', encoding='utf_8') as f:
    
    # iterate through each line (json record) in the file
    for business_json in f:
        
        # convert the json record to a Python dict
        business = json.loads(business_json)
        
        # if this business is not a food, skip to the next one
        if u'Bars' not in business[u'categories']:
            continue
            
        # add the food business id to our food_ids set
        bar_ids.add(business[u'business_id'])


In [39]:
all_ids = [restaurant_ids] + [food_ids] + [bar_ids] 

In [40]:
all_ids = list(itertools.chain.from_iterable(all_ids))
all_ids = frozenset(all_ids) #set gets unique values
len(all_ids)

33926

In [44]:
restaurant_ids = frozenset(restaurant_ids)
len(restaurant_ids)

54618

Reviewers who had written at least 30 reviews were targeted for analysis to provide enough data for clustering and testing thefinal model

In [14]:
#load reviewer list
reviewer_30 =  pd.read_csv('reviewer_30.csv')

In [15]:
user_ids = frozenset(reviewer_30['user_id'])

In [16]:
len(user_ids)

197837

In [31]:
#list reviewers in subsetted list
%%time

review_count = 0
 # create & open a new file in write mode
with codecs.open('review_user_rest_subset.txt', 'w', encoding='utf_8') as review_txt_file:

        # open the existing review json file
    with codecs.open('yelp_academic_dataset_review.json', encoding='utf_8') as review_json_file:

            # loop through all reviews in the existing file and convert to dict
        for review_json in review_json_file:
            review = json.loads(review_json)

                # if this review is not in predetermined business types, skip to the next one
            if review[u'business_id'] not in restaurant_ids:
                continue
                
                # if this review is not written by user with at least 20 reviews, skip to the next one
            if review[u'user_id'] not in user_ids:
                continue
            

                # write the review as a line in the new file
                # escape newline characters in the original review text
            review_txt_file.write(str(review[u'user_id']) + '\n')
            review_count += 1
print('''Text from {:,} subset users
              written to the new txt file.'''.format(review_count))


Text from 1,570,963 subset users
              written to the new txt file.
CPU times: user 2min 10s, sys: 5.36 s, total: 2min 15s
Wall time: 2min 15s


In [32]:
#list number of starts (rating) in subsetted list
%%time

review_count = 0
 # create & open a new file in write mode
with codecs.open('review_business_rest_subset.txt', 'w', encoding='utf_8') as review_txt_file:

        # open the existing review json file
    with codecs.open('yelp_academic_dataset_review.json', encoding='utf_8') as review_json_file:

            # loop through all reviews in the existing file and convert to dict
        for review_json in review_json_file:
            review = json.loads(review_json)

                # if this review is not in predetermined business types, skip to the next one
            if review[u'business_id'] not in restaurant_ids:
                continue
                
                # if this review is not written by user with at least 20 reviews, skip to the next one
            if review[u'user_id'] not in user_ids:
                continue
            

                # write the review as a line in the new file
                # escape newline characters in the original review text
            review_txt_file.write(str(review[u'business_id']) + '\n')
            review_count += 1
print('''Text from {:,} subset business
              written to the new txt file.'''.format(review_count))

Text from 1,570,963 subset business
              written to the new txt file.
CPU times: user 2min 1s, sys: 4.55 s, total: 2min 5s
Wall time: 2min 5s


In [33]:
#list reviews in subsetted list
%%time

review_count = 0
 # create & open a new file in write mode
with codecs.open('review_text_rest_subset.txt', 'w', encoding='utf_8') as review_txt_file:

        # open the existing review json file
    with codecs.open('yelp_academic_dataset_review.json', encoding='utf_8') as review_json_file:

            # loop through all reviews in the existing file and convert to dict
        for review_json in review_json_file:
            review = json.loads(review_json)

                # if this review is not in predetermined business types, skip to the next one
            if review[u'business_id'] not in restaurant_ids:
                continue
                
                # if this review is not written by user with at least 20 reviews, skip to the next one
            if review[u'user_id'] not in user_ids:
                continue
            
            if review[u'text'] in '':
                continue
            

                # write the review as a line in the new file
                # escape newline characters in the original review text
            review_txt_file.write(str(review[u'text'].replace('\n', ' ').replace('\r', ' ').replace('\\', ' ')) + ' ' + '\n')
            review_count += 1
print('''Text from {:,} subset reviews
              written to the new txt file.'''.format(review_count))


Text from 1,570,963 subset reviews
              written to the new txt file.
CPU times: user 2min 8s, sys: 9.37 s, total: 2min 17s
Wall time: 2min 21s


### This section describes how to subset the review dataset based on a threshold for how many reviews they have written

get users who wrote many reviews, DO NOT RUN AGAIN unless changing threshold, read in from csv file

In [3]:
yelp_user = pd.read_json('yelp_academic_dataset_user.json', lines = True)

In [4]:
yelp_user.head()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,4.67,0,0,0,0,0,0,0,0,1,...,0,[],0,"[cvVMmlU1ouS3I5fhutaryQ, nj6UZ8tdGo8YJ9lUMTVWN...",0,Johnny,8,0,oMy_rEb0UBEmMlu-zcxnoQ,2014-11-03
1,3.70,0,0,0,0,0,0,0,0,0,...,0,[],0,"[0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gp...",0,Chris,10,0,JJ-aSuM4pCFPdkfoZ34q0Q,2013-09-24
2,2.00,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Tiffy,1,0,uUzsFQn_6cXDh6rPNGbIFA,2017-03-02
3,4.67,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Mark,6,0,mBneaEEH5EMyxaVyqS-72A,2015-03-13
4,4.67,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Evelyn,3,0,W5mJGs-dcDWRGEhAzUYtoA,2016-09-08


In [4]:
yelp_user.shape

(1326101, 22)

In [5]:
yelp_u_sub = yelp_user[yelp_user['review_count'].apply(lambda x: x >=30)].sort_values(['review_count'], ascending=False)

In [8]:
#write to file to use later
yelp_u_sub.to_csv('reviewer_30.csv', index=False, header=True)